In [1]:
# 1번 셀

# 패키지 수입 
import numpy as np
from  keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from time import time

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.metrics import confusion_matrix, f1_score

In [2]:
# 2번 셀

# 하이퍼 파라미터 설정 : RNN 설계도 참고하면 이해가 수월하다.
MY_WORDS = 5000    # 사전 안에 단어 수 <----------- 무슨 사전인가? 
MY_LEN = 200        # 영화평 길이 
MY_EMBED = 32      # 임베딩 차원
MY_HIDDEN = 64     # LSTM층 차원

MY_EPOCH = 10      # 반복 학습 수 
MY_BATCH = 200     # 한번에 처리하는 데이터 수

In [3]:
# 3번 셀

# 데이터 불러오기 : keras가 사분할 해놓은 데이터 
# 총 5만개 데이터. 학습용 2.5만개, 평가용 2.5만개.
(X_train, Y_train), (X_test, Y_test) = imdb.load_data(num_words=MY_WORDS)

print(type(X_train))
print(X_train.shape)  # (25000, ) : 1차원 데이터. Day-6 판서 참고하시오.
print(Y_train.shape)  # 감성 : 0이면 부정, 1이면 긍정 -> 이진분류문제

print('0번학습용 입력 데이터(영화평) : ', X_train[0]) # 단어가 아닌 숫자 -> Token 처리된 (정수)영화평 -> 정수를 단어로 전환하기 
print('0번학습용 출력 데이터(감성평) : ', Y_train[0])

<class 'numpy.ndarray'>
(25000,)
(25000,)
0번학습용 입력 데이터(영화평) :  [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]
0번학습용 출력 데이터

RNN으로 영화평 감성 분석하기 2022년 1월 10일 김이룸

In [4]:
# 4번 셀

# 단어를 정수로 전환하기
word_to_id = imdb.get_word_index()  # 단어들의 사전, 단어를 입력하면  정수 인덱스(id)를 반환하는 imdb 함수
print('the의 token(id) : ', word_to_id['the'])
print('and의 token(id) : ', word_to_id['and'])


# 정수를 단어로 전환하기
id_to_word = {} 
for word, id in word_to_id.items():
  id_to_word[id] = word

print('Token 1의 단어(word) : ',  id_to_word[1])
print('Token 2의 단어(word) : ',  id_to_word[2])

the의 token(id) :  1
and의 token(id) :  2
Token 1의 단어(word) :  the
Token 2의 단어(word) :  and


In [5]:
# 5번 셀

# 영화평(X_train[i]) 단어로 전환하기 : id-3 & get함수로 음수(0-3, 1-3, 2-3)처리
# dictionary.get() : dict의 Key로 Value얻기
def decode(review):
  output = []
  for i in review:
    word = id_to_word.get(i-3, '???')
    output.append(word)
  print(output)

decode(X_train[0])
print(Y_train[0])   # 1 : pos review

decode(X_train[1])
print(Y_train[1])   # 0 : neg review

['???', 'this', 'film', 'was', 'just', 'brilliant', 'casting', 'location', 'scenery', 'story', 'direction', "everyone's", 'really', 'suited', 'the', 'part', 'they', 'played', 'and', 'you', 'could', 'just', 'imagine', 'being', 'there', 'robert', '???', 'is', 'an', 'amazing', 'actor', 'and', 'now', 'the', 'same', 'being', 'director', '???', 'father', 'came', 'from', 'the', 'same', 'scottish', 'island', 'as', 'myself', 'so', 'i', 'loved', 'the', 'fact', 'there', 'was', 'a', 'real', 'connection', 'with', 'this', 'film', 'the', 'witty', 'remarks', 'throughout', 'the', 'film', 'were', 'great', 'it', 'was', 'just', 'brilliant', 'so', 'much', 'that', 'i', 'bought', 'the', 'film', 'as', 'soon', 'as', 'it', 'was', 'released', 'for', '???', 'and', 'would', 'recommend', 'it', 'to', 'everyone', 'to', 'watch', 'and', 'the', 'fly', '???', 'was', 'amazing', 'really', 'cried', 'at', 'the', 'end', 'it', 'was', 'so', 'sad', 'and', 'you', 'know', 'what', 'they', 'say', 'if', 'you', 'cry', 'at', 'a', 'film

In [6]:
# 6번 셀

# 영화평 길이 확인 : 천차만별
for i in range(5):
  print(i, '번 영화평 길이 : ', len(X_train[i]))


# 영화평(X_train, X_test) 길이 통일 : pad_sequence 함수 사용
# truncating='pre' : 앞부분 자르기
# truncating='post' : 뒷부분 자르기
# padding='pre' : 앞부분에 패딩처리. 앞부분을 (CNN padding='same'처럼)0으로 채우는 것.
# padding='post' : 뒷부분에 패딩처리. 뒷부분을 0으로 채우는 것.
# maxlen : 통일할 영화평의 길이 지정 -> 긴거는 자르고, 짧은 거는 (0으로) 채우기
X_train = pad_sequences(X_train,
                        truncating='pre',
                        padding='pre',
                        maxlen=MY_LEN)

X_test = pad_sequences(X_test,
                      truncating='pre',
                      padding='pre',
                      maxlen=MY_LEN)


# 영화평 길이 확인 
for i in range(5):
  print(i, '번 영화평 길이 : ', len(X_train[i]))

0 번 영화평 길이 :  218
1 번 영화평 길이 :  189
2 번 영화평 길이 :  141
3 번 영화평 길이 :  550
4 번 영화평 길이 :  147
0 번 영화평 길이 :  200
1 번 영화평 길이 :  200
2 번 영화평 길이 :  200
3 번 영화평 길이 :  200
4 번 영화평 길이 :  200


In [7]:
# 7번 셀

# 데이터 모양 확인 : RNN 설계도 참고
# 입력 데이터는 1차원에서 2차원으로 변함. 길이 통일시키면서 변함.
print('학습용 입력 데이터 모양 : ', X_train.shape )
print('학습용 출력 데이터 모양 : ', Y_train.shape )
print('평가용 입력 데이터 모양 : ', X_test.shape )
print('평가용 출력 데이터 모양 : ', Y_test.shape )

학습용 입력 데이터 모양 :  (25000, 200)
학습용 출력 데이터 모양 :  (25000,)
평가용 입력 데이터 모양 :  (25000, 200)
평가용 출력 데이터 모양 :  (25000,)


In [8]:
# 8번 셀

# RNN 구현 : 입력은 단어 80개, 출력은 [0,1] 확률
model = Sequential()

# 임베딩층 add
model.add(Embedding(input_dim=MY_WORDS,
                    output_dim=MY_EMBED,
                    input_length=MY_LEN))

# LSTM층 add
model.add(LSTM(units=MY_HIDDEN))

# 출력층 add
model.add(Dense(units=1,
                activation='sigmoid'))

# RNN 요약
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 32)           160000    
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 184,897
Trainable params: 184,897
Non-trainable params: 0
_________________________________________________________________


In [9]:
# 9번 셀

# RNN 학습
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

print('학습 시작')
begin = time()

model.fit(X_train,
          Y_train,
          epochs=MY_EPOCH,
          batch_size=MY_BATCH,
          verbose=1)

end = time()
print('총 학습 시간 : ', end - begin)

학습 시작
Epoch 1/10
125/125 [==============================] - 9s 54ms/step - loss: 0.5790 - acc: 0.6874
Epoch 2/10
125/125 [==============================] - 7s 53ms/step - loss: 0.2950 - acc: 0.8804
Epoch 3/10
125/125 [==============================] - 7s 55ms/step - loss: 0.2429 - acc: 0.9082
Epoch 4/10
125/125 [==============================] - 7s 55ms/step - loss: 0.2103 - acc: 0.9186
Epoch 5/10
125/125 [==============================] - 7s 54ms/step - loss: 0.1823 - acc: 0.9320
Epoch 6/10
125/125 [==============================] - 7s 55ms/step - loss: 0.1670 - acc: 0.9384
Epoch 7/10
125/125 [==============================] - 7s 54ms/step - loss: 0.1559 - acc: 0.9415
Epoch 8/10
125/125 [==============================] - 7s 55ms/step - loss: 0.1358 - acc: 0.9500
Epoch 9/10
125/125 [==============================] - 7s 54ms/step - loss: 0.1227 - acc: 0.9566
Epoch 10/10
125/125 [==============================] - 7s 54ms/step - loss: 0.1116 - acc: 0.9604
총 학습 시간 :  70.56662940979004


In [10]:
# 10번 셀

# RNN 평가
score = model.evaluate(X_test,
                       Y_test)

print('최종 정확도 : ', score[1])

782/782 [==============================] - 12s 15ms/step - loss: 0.4054 - acc: 0.8440
최종 정확도 :  0.8439599871635437


In [13]:
# 11번 셀

# RNN 예측
pred = model.predict(X_test)

# 평가용 데이터 샘플 예측
print('영화평 : ', decode(X_test[1]))
print('정답 : ', Y_test[1])
print('예측 : ', pred[1]) # [0,1] 사이 확률값 반환 -> 0.5보다 크면 1(긍정), 0.5보다 작으면 0(긍정)

['psychological', '???', "it's", 'very', 'interesting', 'that', 'robert', '???', 'directed', 'this', 'considering', 'the', 'style', 'and', 'structure', 'of', 'his', 'other', 'films', 'still', 'the', '???', '???', 'audio', 'style', 'is', 'evident', 'here', 'and', 'there', 'i', 'think', 'what', 'really', 'makes', 'this', 'film', 'work', 'is', 'the', 'brilliant', 'performance', 'by', '???', 'dennis', "it's", 'definitely', 'one', 'of', 'her', 'darker', 'characters', 'but', 'she', 'plays', 'it', 'so', 'perfectly', 'and', 'convincingly', 'that', "it's", 'scary', 'michael', 'burns', 'does', 'a', 'good', 'job', 'as', 'the', '???', 'young', 'man', 'regular', '???', 'player', 'michael', 'murphy', 'has', 'a', 'small', 'part', 'the', '???', 'moody', 'set', 'fits', 'the', 'content', 'of', 'the', 'story', 'very', 'well', 'in', 'short', 'this', 'movie', 'is', 'a', 'powerful', 'study', 'of', '???', 'sexual', '???', 'and', 'desperation', 'be', 'patient', '???', 'up', 'the', 'atmosphere', 'and', 'pay', 

In [14]:
# 12번 셀

# 확률 결과를 이진수로 전환 : 혼동행렬 만들기 위함
print(pred)
pred = (pred > 0.5)
print(pred)

# 혼동행렬
print(confusion_matrix(Y_test,
                       pred))

# F1 점수
print(f1_score(Y_test,
               pred,
               average='micro'))

[[0.12727448]
 [0.9853757 ]
 [0.6273767 ]
 ...
 [0.09291574]
 [0.0059434 ]
 [0.61575145]]
[[False]
 [ True]
 [ True]
 ...
 [False]
 [False]
 [ True]]
[[10927  1573]
 [ 2328 10172]]
0.84396
